# loading libraries 


importing all necessary libs and packages for the prediction , in this work I will create embeddings features from the text using A Bert model , and uses them as predictors to classify sentiments wich are our labels , this work is a luti classes classification since we have three labels 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report 
import os 
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import pickle 
import random 

fixing the seed to avoid different results in the training (in case of rerun )

In [2]:
SEED=42

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
seed_everything(SEED)    

Downoald a bert necessary library for text tokenzation 

In [4]:
!wget https://raw.githubusercontent.com/kpe/bert-for-tf2/master/bert/tokenization/bert_tokenization.py -O tokenization.py


--2021-03-31 11:12:50--  https://raw.githubusercontent.com/kpe/bert-for-tf2/master/bert/tokenization/bert_tokenization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13552 (13K) [text/plain]
Saving to: ‘tokenization.py’

tokenization.py     100%[===================>]  13.23K  --.-KB/s    in 0.001s  

2021-03-31 11:12:51 (18.3 MB/s) - ‘tokenization.py’ saved [13552/13552]



# librairies for loading BERT MODEL With only URL LINK 

In [5]:
import tokenization
import tensorflow_hub as hub

# LOAD TRAING DATA  AND BUILDING THE MODEL 

In [6]:
train=pd.read_csv('../input/atrdius/training.tsv',sep='\t')

In [7]:
train.head(10)

,label,sentence
0,positive,"In December alone , the members of the Lithuan..."
1,neutral,Exel wants to serve its industrial customers w...
2,neutral,"Panostaja , headquartered in Tampere , Finland..."
3,neutral,The reductions will be implemented mainly in t...
4,neutral,`` Because we 're a pension insurance company ...
5,positive,`` I am very pleased and proud of our performa...
6,negative,All the ferries had run into trouble just outs...
7,positive,`` The Government has had a very professional ...
8,neutral,The company also said that its board of direct...
9,positive,"Kesko 's car import and retailing business , V..."


In [8]:
encoder = LabelEncoder()
train['target'] = encoder.fit_transform(train['label'])

the function below is used to convert text (our input ) to  BERT ENCODER  inputs 

In [9]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [10]:
max_len=100
num_classes=3


# Load bert pretrained model , from tensor flow hub 

In [11]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable = True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [12]:
tokenizer = tokenization.FullTokenizer(vocab_file = vocab_file, do_lower_case = do_lower_case)

In [13]:
input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

In [14]:
pooled_output,sequence_output=bert_layer([input_word_ids, input_mask, segment_ids])

clf_output = sequence_output[:, 0, :]


In [15]:
#x = tf.keras.layers.Dropout(rate=0.1)(pooled_output)
#output = tf.keras.layers.Dense(num_classes, activation='softmax', name='classifier')(x)

In [16]:
model = tf.keras.models.Model(inputs = [input_word_ids, input_mask, segment_ids], outputs = [clf_output])

In [17]:
text_train= bert_encode(train['sentence'], tokenizer, max_len = max_len)

In [18]:
#x_val= bert_encode(x_val, tokenizer, max_len = max_len)

In [19]:
#y_train = tf.keras.utils.to_categorical(y_train.values)
#y_val = tf.keras.utils.to_categorical(y_val.values)

In [20]:
VERBOSE = 1
LR = 0.00001
EPOCHS =60
BATCH_SIZE=32


In [21]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr = LR),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.Accuracy()])

# create embedinng 

# As we will use BERT only for features generation , we cut the model at the last encoder layer , and we use the output from there 

In [22]:
embeddings = model.predict(text_train)

splitting data for training 

In [23]:
x_train, x_val, y_train, y_val = train_test_split(embeddings, train['target'], shuffle = True, random_state = SEED, test_size = 0.1)

In [24]:
embeddings.shape

(2951, 1024)

# classification part :

In this part we will use the training data passed throught the model to train a classifier 

# classifier 1:LGBM 

In [25]:
import lightgbm as lgb

In [26]:
def naming(x):
    return 'col_'+ str(x)

columns= list(map(naming ,range(embeddings.shape[1])))

In [27]:
train_df=pd.DataFrame(x_train,columns=columns)

val_df=pd.DataFrame(x_val,columns=columns)

In [28]:
train_df.shape

(2655, 1024)

In [29]:
val_df.shape

(296, 1024)

In [30]:
y_train = y_train.values
y_val = y_val.values

In [31]:
lgb_train = lgb.Dataset(train_df, y_train)
lgb_val = lgb.Dataset(val_df, y_val)

In [32]:
params = {
        
    
    
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': num_classes,
    'metric': ['multi_error'],
    "learning_rate": 0.2,
     "num_leaves": 60,
     "max_depth": 10,
     "feature_fraction": 0.45,
     "bagging_fraction": 0.3,
     "reg_alpha": 0.15,
     "reg_lambda": 0.15,
#      "min_split_gain": 0,
      "min_child_weight": 0
    }

lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=5)





[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 2655, number of used features: 1024
[LightGBM] [Info] Start training from score -2.112759
[LightGBM] [Info] Start training from score -0.511454
[LightGBM] [Info] Start training from score -1.274851
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's multi_error: 0.243243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	valid_0's multi_error: 0.172297
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	valid_0's multi_error: 0.168919


# TESTING :
in this part I test the model using the test data , after preapring it 

In [33]:
test=pd.read_csv('../input/atrdius/testing.tsv',sep='\t')
text_test= bert_encode(test['sentence'], tokenizer, max_len = max_len)
test_embeddings = model.predict(text_test)
test_df=pd.DataFrame(test_embeddings,columns=columns)

In [34]:
encoder = LabelEncoder()
test['target'] = encoder.fit_transform(test['label'])

In [35]:
Y_predict=lgbm_model.predict(test_df ) 
label_predict=encoder.classes_[np.argmax(Y_predict,axis=1)] 
print(classification_report(test['label'],label_predict,digits=4))

              precision    recall  f1-score   support

    negative     0.7642    0.6104    0.6787       154
     neutral     0.8060    0.9501    0.8721       761
    positive     0.8211    0.5755    0.6767       351

    accuracy                         0.8049      1266
   macro avg     0.7971    0.7120    0.7425      1266
weighted avg     0.8051    0.8049    0.7944      1266



# SVM CLASSIFIER 

Data standarization with builtin sckit learn function Scaler

In [36]:

scaler = StandardScaler()
scaler.fit(embeddings)
embeddings_scaled=scaler.transform(embeddings)
test_scaled=scaler.transform(test_embeddings)

In [37]:

SVM = svm.SVC(C=0.5, kernel='sigmoid', degree=3, gamma='auto',probability=True)
SVM.fit(embeddings_scaled,train['target'].values)
# predict the labels on testing data 
predictions_SVM = SVM.predict(test_scaled)
svm_label_predict=encoder.classes_[predictions_SVM]

# Use accuracy_score function to get the accuracy
print(classification_report(test['label'],svm_label_predict,digits=4))



              precision    recall  f1-score   support

    negative     0.7329    0.6948    0.7133       154
     neutral     0.8268    0.9409    0.8801       761
    positive     0.8386    0.6068    0.7041       351

    accuracy                         0.8183      1266
   macro avg     0.7994    0.7475    0.7659      1266
weighted avg     0.8186    0.8183    0.8111      1266



# blend of SVM and LGBM  for prediction 

In [45]:
svm_predict_proba=SVM.predict_proba(test_scaled)
Global_predict=(0.4*Y_predict+0.6*svm_predict_proba)
Global_label_predict=encoder.classes_[np.argmax(Global_predict,axis=1)] 
print(classification_report(test['label'],Global_label_predict,digits=4))

              precision    recall  f1-score   support

    negative     0.7519    0.6494    0.6969       154
     neutral     0.8205    0.9488    0.8800       761
    positive     0.8419    0.6068    0.7053       351

    accuracy                         0.8175      1266
   macro avg     0.8047    0.7350    0.7607      1266
weighted avg     0.8181    0.8175    0.8093      1266



# saving models that will be used in the api 

saving the lgbm model

In [39]:
lgbm_model.save_model(f'lgbm_model.lgb')

saving scaler 

In [40]:
with open('scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)

saving SVM as pickle 

In [41]:
with open('SVM.pickle', 'wb') as f:
    pickle.dump(SVM, f)